In [1]:
%load_ext autoreload
%autoreload 2

import os 
# os.environ['R_HOME']= r'C:\Users\tomha\miniconda3\envs\octagon_analysis\lib\R'
os.environ['R_HOME']= r'D:\Users\Tom\miniconda3\envs\octagon_analysis\lib\R'
# os.environ['R_HOME']= '/home/tom/miniconda3/envs/octagon_analysis/lib/R'

import rpy2

import rpy2.robjects as robjects
print(robjects.r('R.version.string'))

import parse_data.prepare_data as prepare_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import globals
import data_strings
import data_extraction.get_indices as get_indices
import analysis.wall_visibility_and_choice as wall_visibility_and_choice
from trajectory_analysis import trajectory_vectors
from plotting import plot_octagon
import parse_data.identify_filepaths as identify_filepaths 
from data_extraction.trial_list_filters import filter_trials_other_visible
from analysis import opponent_visibility
from ipywidgets import IntProgress
from IPython.display import display
import time
from pymer4.models import Lmer



[1] "R version 4.1.3 (2022-03-10)"



### load data

In [2]:
import pickle

analysis_dir = os.path.join('..', 'data')
analysis_file = 'analysis_results_2levelsforFirstSeenWall.pkl'
filename = os.path.join(analysis_dir, analysis_file)
# load the analysis results
with open(filename, 'rb') as f:
    analysis_results = pickle.load(f)

#### Populate a dataframe, with a row for each trial, and fields for regressors (only including trials with fully-populated regressors)

#### Social df

In [3]:
glm_df_social = pd.DataFrame()

for session_id, players in analysis_results.items():
    for player_id in players:
        
        # take each filtered_regressor array and fill the relevant df field for this player
        player_data = analysis_results[session_id][player_id]['social']['regressors']
        choice = analysis_results[session_id][player_id]['social']['dependent']['choice']
        opponent_player_id = 1 if player_id == 0 else 1
        opponent_player_data = analysis_results[session_id][opponent_player_id]['social']['regressors']
        df_player = pd.DataFrame(
                    {
                        "SessionID" : session_id,
                        "PlayerID" : player_id,
                        "GlmPlayerID" : session_id*2 + player_id,
                        "ChooseHigh" : choice,
                        "WallSep" : player_data['wall_sep'],
                        "FirstSeenWall" : player_data['first_seen'],
                        "D2H" : player_data['d2h'],
                        "D2L" : player_data['d2l'],
                        "OpponentVisible" : player_data['opponent_visible'],
                        "OpponentFirstSeenWall" : player_data['first_seen_opponent'],
                        "OpponentD2H" : player_data['d2h_opponent'],
                        "OpponentD2L" : player_data['d2l_opponent']
                        
                    }
        )


        # append this smaller dataframe to the the full dataframe
        glm_df_social = pd.concat([glm_df_social, df_player], ignore_index=True)



glm_df_social["FirstSeenWall"] = glm_df_social["FirstSeenWall"].astype(str).astype("category")
glm_df_social["OpponentFirstSeenWall"] = glm_df_social["OpponentFirstSeenWall"].astype(str).astype("category")

glm_df_social["WallSep"] = glm_df_social["WallSep"].astype(str).astype("category")

#### solo-social combined df

In [4]:
glm_df_solo_social = pd.DataFrame()

for session_id, players in analysis_results.items():
    for player_id in players:
        
        # take each filtered_regressor array and fill the relevant df field for this player
        player_data_solo = analysis_results[session_id][player_id]['solo']['regressors']
        player_data_social = analysis_results[session_id][player_id]['social']['regressors']
        choice_solo = analysis_results[session_id][player_id]['solo']['dependent']['choice']
        choice_social = analysis_results[session_id][player_id]['social']['dependent']['choice']
        df_player = pd.DataFrame(
                    {
                        "SessionID" : session_id,
                        "PlayerID" : player_id,
                        "GlmPlayerID" : session_id*2 + player_id,
                        "ChooseHigh" : np.concatenate([choice_solo, choice_social]),
                        "WallSep" :  np.concatenate([player_data_solo['wall_sep'], player_data_social['wall_sep']]),
                        "FirstSeenWall" : np.concatenate([player_data_solo['first_seen'], player_data_social['first_seen']]),
                        "D2H" : np.concatenate([player_data_solo['d2h'], player_data_social['d2h']]),
                        "D2L" : np.concatenate([player_data_solo['d2l'], player_data_social['d2l']]),
                        "SocialContext" : np.concatenate([np.ones(player_data_solo["wall_sep"].shape[0]) - 1, np.ones(player_data_social["wall_sep"].shape[0])]) # 0 for solo, 1 for social
                    }
        )

        # append this smaller dataframe to the the full dataframe
        glm_df_solo_social = pd.concat([glm_df_solo_social, df_player], ignore_index=True)


glm_df_solo_social["FirstSeenWall"] = glm_df_solo_social["FirstSeenWall"].astype(str).astype("category")
glm_df_solo_social["WallSep"] = glm_df_solo_social["WallSep"].astype(str).astype("category")

#### Solo df

In [5]:
glm_df_solo = pd.DataFrame()

for session_id, players in analysis_results.items():
    for player_id in players:
        
        # take each filtered_regressor array and fill the relevant df field for this player
        player_data = analysis_results[session_id][player_id]['solo']['regressors']
        choice = analysis_results[session_id][player_id]['solo']['dependent']['choice']
        df_player = pd.DataFrame(
                    {
                        "SessionID" : session_id,
                        "PlayerID" : player_id,
                        "GlmPlayerID" : session_id*2 + player_id,
                        "ChooseHigh" : choice,
                        "WallSep" : player_data['wall_sep'],
                        "FirstSeenWall" : player_data['first_seen'],
                        "D2H" : player_data['d2h'],
                        "D2L" : player_data['d2l']
                    }
        )

        # append this smaller dataframe to the the full dataframe
        glm_df_solo = pd.concat([glm_df_solo, df_player], ignore_index=True)


glm_df_solo["FirstSeenWall"] = glm_df_solo["FirstSeenWall"].astype(str).astype("category")
glm_df_solo["WallSep"] = glm_df_solo["WallSep"].astype(str).astype("category")



In [6]:
# Random indices should only be generated once, so we can use the same random indices for all model types
# to ensure that the models are comparable
# Store and retrieve these indices from a .pickle file
def generate_random_indices(df, n=400, random_seed=17):
    ''' select n random indices from the DataFrame df, ensuring that the indices are valid (i.e., not NaN) and that they are unique. '''
    
    # Change "nan" strings to np.nan for dropping indices
    df_copy = df.copy()
    df_copy["FirstSeenWall"].replace("nan", np.nan, inplace=True)
    valid_indices = df_copy.dropna().index.tolist()
    print("Valid indices:", len(valid_indices))
    print("Rows with NaN values in df_shuffle:")
    print(df_copy[df_copy.isna().any(axis=1)].shape[0])
    
    # randomly generate n integers between 0 and the length of the DataFrame, without replacement
    random_indices = np.random.choice(valid_indices, size=n, replace=False)

    return random_indices



In [7]:
import pickle

file_dir = os.path.join('..', 'data')
filenames = ['random_indices_solo.pkl', 'random_indices_solo_social.pkl', 'random_indices_social.pkl']

random_indices_solo = generate_random_indices(glm_df_solo, n=400, random_seed=17)
random_indices_solo_social = generate_random_indices(glm_df_solo_social, n=400, random_seed=17)
random_indices_social = generate_random_indices(glm_df_social, n=400, random_seed=17)



# # Save random_indices to a file
# file_dir = os.path.join('..', 'data')
# filename = filenames[2]
# filepath = os.path.join(file_dir, filename)
# with open(filepath, 'wb') as f:
#     pickle.dump(random_indices_social, f)


filename = filenames[0]
filepath = os.path.join(file_dir, filename)
# load the analysis results
with open(filepath, 'rb') as f:
    random_indices_solo = pickle.load(f)


filename = filenames[1]
filepath = os.path.join(file_dir, filename)
# load the analysis results
with open(filepath, 'rb') as f:
    random_indices_solo_social = pickle.load(f)



filename = filenames[2]
filepath = os.path.join(file_dir, filename)
# load the analysis results
with open(filepath, 'rb') as f:
    random_indices_social = pickle.load(f)



Valid indices: 3441
Rows with NaN values in df_shuffle:
1506
Valid indices: 7461
Rows with NaN values in df_shuffle:
5258
Valid indices: 4020
Rows with NaN values in df_shuffle:
3752


C:\Users\Tom\AppData\Local\Temp\ipykernel_11652\3603952172.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_copy["FirstSeenWall"].replace("nan", np.nan, inplace=True)
C:\Users\Tom\AppData\Local\Temp\ipykernel_11652\3603952172.py:9: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_copy["FirstSeenWall"].replace(

In [8]:
from contextlib import redirect_stdout


def generate_leave_one_out_dataframes(df):
    
    # # randomise the order of the rows
    # df_shuffle = df.sample(frac=1, random_state=17).reset_index(drop=True)
    # print(f" df_shuffle type: {type(df_shuffle)}")

    # create lists to store the DataFrames
    dfs_with_row_removed = []
    dfs_with_removed_row = []

    # iterate through each row index in the DataFrame
    for i in range(len(df)):
        # create a DataFrame with one row removed
        df_without_row = df.drop(index=i).reset_index(drop=True)
        dfs_with_row_removed.append(df_without_row)
        
        # create a DataFrame with only the removed row
        df_with_removed_row = df.iloc[[i]].reset_index(drop=True)
        dfs_with_removed_row.append(df_with_removed_row)

    # Now you have two lists:
    # 1. dfs_with_row_removed: DataFrames with one row removed
    # 2. dfs_with_removed_row: DataFrames containing only the removed rows

    return dfs_with_row_removed, dfs_with_removed_row


def select_data_for_models(random_indices, dfs_with_row_removed, dfs_with_removed_row, n=5, random_seed=None):

    if random_seed is not None:
        np.random.seed(random_seed)

    # restrict the dfs_with_row_removed and dfs_with_removed_row lists to only the randomly selected indices
    dfs_with_row_removed_sampled = [dfs_with_row_removed[i] for i in random_indices]
    dfs_with_removed_row_sampled = [dfs_with_removed_row[i] for i in random_indices]

    print(dfs_with_removed_row_sampled)

    return dfs_with_row_removed_sampled, dfs_with_removed_row_sampled

def fit_models(dfs_with_row_removed_sampled, model_formula):
    
    models = []
    max_count = len(dfs_with_row_removed_sampled)
    f = IntProgress(min=0, max=max_count, description='Fitting models')
    display(f)

    # Suppress the output of the models fitting process
    with open(os.devnull, 'w') as fnull:
        with redirect_stdout(fnull):
            for i, df in enumerate(dfs_with_row_removed_sampled):
                model = Lmer(model_formula, data=df, family='binomial')
                model.fit()
                models.append(model)
                print(f"Model {i} fit with {len(df)} rows")
                f.value += 1


    
    return models

def calculate_predictions(models, original_df_size, dfs_with_removed_row_sampled, random_indices):
    
    predictions = np.full(len(dfs_with_removed_row_sampled), np.nan)
    predictions_maintained_index = np.full(original_df_size, np.nan)
    for i, model in enumerate(models):
        # get the row that was removed for this model
        removed_row = dfs_with_removed_row_sampled[i]
        
        # get the prediction for this row
        prediction = model.predict(removed_row, skip_data_checks=True, verify_predictions=False)
        
        # assign the prediction to the correct index in the predictions array
        predictions_maintained_index[random_indices[i]] = prediction[0]

        # also assign the prediction to the next index of a new array
        predictions[i] = prediction[0]

    return predictions, predictions_maintained_index

def calculate_likelihoods(df, predictions_maintained_index, random_indices):
    
    # calculate the metric for each prediction
    likelihoods = np.full(len(random_indices), np.nan)
    for i, idx in enumerate(random_indices):
        predicted_output = predictions_maintained_index[idx]
        true_output = df.iloc[idx]['ChooseHigh']
        likelihood = predicted_output**true_output * (1 - predicted_output)**(1 - true_output)
        likelihoods[i] = likelihood

    return likelihoods

def calculate_nll(likelihoods):
    # #### sum the logs of the likelihoods, and take the negative
    summed_log_likelihoods = np.sum(np.log(likelihoods)) 
    nll = -summed_log_likelihoods

    return nll

def save_cross_validation_results(name, model_formula, df, random_indices, predictions, nll):
    ''' Save the cross-validation results to a file. '''
    
    cross_validation_results = {
        "name": name,
        "model_formula": model_formula,
        "dataframe": df,
        "random_indices" : random_indices,
        # "models" : models,
        "predictions" : predictions,
        "nll" : nll
    }

   # Save the cross-validation results to a file
    dir = os.path.join('..', 'data')
    filename = f'CV_results_{name}.pickle'
    filepath = os.path.join(dir, filename)
    with open(filepath, 'wb') as f:
        pickle.dump(cross_validation_results, f)

    print("CV data saved to: ", filepath)

def save_cross_validations_models(name, models):
    ''' Save the cross-validation models to a file. '''
    
    cross_validation_models = {
        "name": name,
        "models" : models
    }

    # Save the cross-validation models to a file
    dir = os.path.join('..', 'models')
    filename = f'CV_models_{name}.pickle'
    filepath = os.path.join(dir, filename)
    with open(filepath, 'wb') as f:
        pickle.dump(cross_validation_models, f)

    print("CV models saved to: ", filepath)



In [14]:
def run_cross_validation(df, model_formula, name, random_indices, n=50, save_results=False, save_models=False, random_seed=None):
    ''' Run leave-one-out cross-validation on the given dataframe and model formula.
        Returns the negative log-likelihood (NLL), fitted models, random indices, 
        predictions, and likelihoods.
        
        Arguments:
        df: DataFrame containing the data for cross-validation.
        model_formula: String representing the model formula for the GLM.
        name: String representing the name for saving the models.
        n: Number of random samples to select for cross-validation.
        save_models: Boolean indicating whether to save the models to file.
        
        Returns:
        nll: Negative log-likelihood of the model.
        models: List of fitted models.
        random_indices: List of random indices used for cross-validation.
        predictions: Array of predictions from the models.
        likelihoods: Array of likelihoods calculated from the predictions. '''
    
    n_rows = df.shape[0]

    # Step 1: Generate leave-one-out dataframes
    dfs_with_row_removed, dfs_with_removed_row = generate_leave_one_out_dataframes(df)

    # Step 2: Select data for models
    (dfs_with_row_removed_sampled,
     dfs_with_removed_row_sampled) = select_data_for_models(random_indices, dfs_with_row_removed, dfs_with_removed_row, n, random_seed=random_seed)

    # Step 3: Fit models
    models = fit_models(dfs_with_row_removed_sampled, model_formula)

    # Step 4: Calculate predictions
    predictions, predictions_maintained_index = calculate_predictions(models, n_rows, dfs_with_removed_row_sampled, random_indices)

    # Step 5: Calculate likelihoods
    likelihoods = calculate_likelihoods(df, predictions_maintained_index, random_indices)

    # Step 6: Calculate NLL
    nll = calculate_nll(likelihoods)

    # Step 7: Save data to file (optional)
    if save_results:
        try:
            save_cross_validation_results(name, model_formula, df, random_indices, predictions, nll)
        except Exception as e:
            print(f"Error saving results: {e}")
            return nll, models, predictions, likelihoods
    
    # Step 8: Save models to file (optional)
    if save_models:
        try:
            save_cross_validations_models(name, models)
        except Exception as e:
            print(f"Error saving models: {e}")
            return nll, models, predictions, likelihoods
    
    return nll, models, predictions, likelihoods

In [10]:
# Count rows with NaN values in any column
nan_rows_count = glm_df_solo.isna().any(axis=1).sum()

# Print the result
print(f"Number of rows with NaN values in glm_df_solo: {nan_rows_count}")

# Optionally, display the rows with NaN values
nan_rows = glm_df_solo[glm_df_solo.isna().any(axis=1)]
print("Rows with NaN values:")
print(nan_rows)

Number of rows with NaN values in glm_df_solo: 0
Rows with NaN values:
Empty DataFrame
Columns: [SessionID, PlayerID, GlmPlayerID, ChooseHigh, WallSep, FirstSeenWall, D2H, D2L]
Index: []


### Solo models

In [11]:
# model_formula = 'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + (1|GlmPlayerID)'
# (nll, models,
#   predictions, likelihoods) = run_cross_validation(glm_df_solo, model_formula,
#                                                     "solo_randomintercepts_400", random_indices_solo, n=400,
#                                                       save_results=True, save_models=True, random_seed=17)

In [ ]:
# model_formula = 'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + (D2L||GlmPlayerID)'
# (nll, models,
#   predictions, likelihoods) = run_cross_validation(glm_df_solo, model_formula,
#                                                     "solo_randomintercepts_randomd2l_400", random_indices_solo, n=400,
#                                                       save_results=True, random_seed=17)

In [ ]:
# model_formula = 'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + WallSep:FirstSeenWall + (1|GlmPlayerID)'
# (nll, models,
#   predictions, likelihoods) = run_cross_validation(glm_df_solo, model_formula,
#                                                     "solo_randomintercepts_randomd2l_lowinteractions_400", random_indices_solo, n=400,
#                                                       save_results=True, random_seed=17)

In [ ]:
# model_formula = 'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + WallSep:FirstSeenWall + D2H:FirstSeenWall + (1|GlmPlayerID)'
# (nll, models,
#   predictions, likelihoods) = run_cross_validation(glm_df_solo, model_formula,
#                                                     "solo_randomintercepts_randomd2l_midinteractions_400", random_indices_solo, n=400,
#                                                       save_results=True, random_seed=17)

In [ ]:
# model_formula = 'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + WallSep:FirstSeenWall + D2H:FirstSeenWall + D2L:FirstSeenWall + (1|GlmPlayerID)'
# (nll, models,
#   predictions, likelihoods) = run_cross_validation(glm_df_solo, model_formula,
#                                                     "solo_randomintercepts_randomd2l_allinteractions_400", random_indices_solo, n=400,
#                                                       save_results=True, random_seed=17)

In [16]:
model_formula = 'ChooseHigh ~ 1 + D2H + D2L + WallSep + (1|GlmPlayerID)'
(nll, models,
  predictions, likelihoods) = run_cross_validation(glm_df_solo, model_formula,
                                                    "solo_randomintercepts_drop-firstseenwall_400", random_indices_solo, n=400,
                                                      save_results=True, random_seed=17)

[   SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H         D2L
0         12         1           25         1.0     2.0           1.0  0.411013869  0.48744693,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H         D2L
0          2         1            5         1.0     1.0           2.0  0.463934495  0.41143346,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L
0          7         0           14         1.0     1.0           2.0  0.390761052  0.672185753,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L
0          5         0           10         1.0     4.0           1.0  0.460135835  0.467986967,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L
0         22         0           44         1.0     4.0           1.0  0.402229131  0.546535013,    SessionID  PlayerID  GlmPlayerID  C

IntProgress(value=0, description='Fitting models', max=400)

d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_v

CV data saved to:  ..\data\CV_results_solo_randomintercepts_drop-firstseenwall_400.pickle


In [ ]:
#### Run all solo models
model_formulas = [
    'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + (1|GlmPlayerID)',
    'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + (D2L||GlmPlayerID)',
    'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + WallSep:FirstSeenWall + (1|GlmPlayerID)',
    'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + WallSep:FirstSeenWall + D2H:FirstSeenWall + (1|GlmPlayerID)',
    'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + WallSep:FirstSeenWall + D2H:FirstSeenWall + D2L:FirstSeenWall + (1|GlmPlayerID)'
]
model_names = [
    "social_randomintercepts_400",
    "social_randomintercepts_randomd2l_400",
    "social_randomintercepts_randomd2l_lowinteractions_400",
    "social_randomintercepts_randomd2l_midinteractions_400",
    "social_randomintercepts_randomd2l_allinteractions_400"
]

results = []

for model_formula, model_name in zip(model_formulas, model_names):
    nll, models, predictions, likelihoods = run_cross_validation(
        glm_df_solo, model_formula, model_name, n=400, save_results=True, random_seed=17
    )
    results.append({
        "model_name": model_name,
        "nll": nll,
        "predictions": predictions,
        "likelihoods": likelihoods
    })

for result in results:
    print(f"Model: {result['model_name']}, NLL: {result['nll']}")

### Solo-Social models

In [11]:
model_formula = 'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + SocialContext + (1 |GlmPlayerID)'
(nll, models,
  predictions, likelihoods) = run_cross_validation(glm_df_solo_social, model_formula,
                                                    "solo-social_randomintercepts_400", random_indices_solo_social, n=400,
                                                      save_results=True, save_models=True, random_seed=17)

[   SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  SocialContext
0         22         0           44         1.0     2.0           1.0  0.536303844  0.214656757            0.0,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall         D2H          D2L  SocialContext
0         23         1           47         0.0     4.0           2.0  0.89771568  0.032469796            0.0,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  SocialContext
0         18         1           37         1.0     4.0           2.0  0.435031559  0.520849718            0.0,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  SocialContext
0         14         0           28         1.0     1.0           1.0  0.296069401  0.282949613            1.0,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  SocialContex

IntProgress(value=0, description='Fitting models', max=400)

/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprec

CV data saved to:  ../data/CV_results_solo-social_randomintercepts_400.pickle
CV models saved to:  ../models/CV_models_solo-social_randomintercepts_400.pickle


In [14]:
model_formula = 'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + SocialContext' \
' + FirstSeenWall:SocialContext + D2H:SocialContext + D2L:SocialContext + (1 |GlmPlayerID)'
(nll, models,
  predictions, likelihoods) = run_cross_validation(glm_df_solo_social, model_formula,
                                                    "solo-social_randomintercepts_interactions_400", random_indices_solo_social, n=400,
                                                      save_results=True, save_models=True, random_seed=17)

[   SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  SocialContext
0         22         0           44         1.0     2.0           1.0  0.536303844  0.214656757            0.0,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall         D2H          D2L  SocialContext
0         23         1           47         0.0     4.0           2.0  0.89771568  0.032469796            0.0,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  SocialContext
0         18         1           37         1.0     4.0           2.0  0.435031559  0.520849718            0.0,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  SocialContext
0         14         0           28         1.0     1.0           1.0  0.296069401  0.282949613            1.0,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  SocialContex

IntProgress(value=0, description='Fitting models', max=400)

/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprec

CV data saved to:  ../data/CV_results_solo-social_randomintercepts_interactions_400.pickle


: 

### Social models

In [ ]:
model_formula = 'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + OpponentVisible + OpponentD2H' \
' + OpponentD2L + OpponentFirstSeenWall + (1|GlmPlayerID)'

(nll, models,
  predictions, likelihoods) = run_cross_validation(glm_df_social, model_formula,
                                                    "social_randomintercepts_opponentvisible_400", random_indices_social, n=400,
                                                      save_results=True, save_models=True, random_seed=17)

[   SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall         D2H          D2L  OpponentVisible OpponentFirstSeenWall  OpponentD2H  OpponentD2L
0         31         1           63         0.0     4.0           2.0  0.43078735  0.510272508                0                   1.0  0.362521694  0.562039199,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  OpponentVisible OpponentFirstSeenWall  OpponentD2H  OpponentD2L
0         21         0           42         1.0     4.0           1.0  0.906108861  0.329014745                0                   1.0  0.893476157  0.131553932,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  OpponentVisible OpponentFirstSeenWall  OpponentD2H  OpponentD2L
0         31         0           62         0.0     2.0           2.0  0.493552107  0.321920475                0                   1.0  0.487531745  0.607921912,    SessionID  PlayerID  GlmPlaye

IntProgress(value=0, description='Fitting models', max=400)

/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
/home/tom/miniconda3/envs/octagon_analysis/lib/python3.12/site-packages/pymer4/models/Lmer.py:733: FutureWarning: DataFrame.applymap has been deprec

CV data saved to:  ../data/CV_results_solo-social_randomintercepts_opponentvisible_400.pickle
CV models saved to:  ../models/CV_models_solo-social_randomintercepts_opponentvisible_400.pickle


In [ ]:
cross_validation_models = {
    "name": name,
    "models" : models
}

# Save the cross-validation models to a file
dir = os.path.join('..', 'models')
filename = f'CV_models_{name}.pickle'
filepath = os.path.join(dir, filename)
with open(filepath, 'wb') as f:
    pickle.dump(cross_validation_models, f)

print("CV models saved to: ", filepath)


In [ ]:
model_formula = 'ChooseHigh ~ 1 + D2H + D2L + FirstSeenWall + WallSep + OpponentD2H' \
' + OpponentD2L + OpponentFirstSeenWall + (1|GlmPlayerID)'

(nll, models,
  predictions, likelihoods) = run_cross_validation(glm_df_social, model_formula,
                                                    "social_randomintercepts_400", random_indices_social, n=400,
                                                      save_results=True, save_models=True, random_seed=17)

[   SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall         D2H          D2L  OpponentVisible OpponentFirstSeenWall  OpponentD2H  OpponentD2L
0         31         1           63         0.0     4.0           2.0  0.43078735  0.510272508                0                   1.0  0.362521694  0.562039199,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  OpponentVisible OpponentFirstSeenWall  OpponentD2H  OpponentD2L
0         21         0           42         1.0     4.0           1.0  0.906108861  0.329014745                0                   1.0  0.893476157  0.131553932,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  OpponentVisible OpponentFirstSeenWall  OpponentD2H  OpponentD2L
0         31         0           62         0.0     2.0           2.0  0.493552107  0.321920475                0                   1.0  0.487531745  0.607921912,    SessionID  PlayerID  GlmPlaye

IntProgress(value=0, description='Fitting models', max=400)

d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_v

CV data saved to:  ..\data\CV_results_solo-social_randomintercepts_400.pickle


FileNotFoundError: [Errno 2] No such file or directory: '..\\models\\CV_models_solo-social_randomintercepts_400.pickle'

In [15]:
model_formula = 'ChooseHigh ~ D2H + D2L + FirstSeenWall + WallSep + OpponentD2H' \
' + OpponentD2L + OpponentFirstSeenWall + WallSep:FirstSeenWall + D2L:FirstSeenWall + D2H:FirstSeenWall +  (1 | GlmPlayerID)'

(nll, models,
  predictions, likelihoods) = run_cross_validation(glm_df_social, model_formula,
                                                    "social_randomintercepts_allinteractions_400", random_indices_social, n=400,
                                                      save_results=True, save_models=True, random_seed=17)

[   SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall         D2H          D2L  OpponentVisible OpponentFirstSeenWall  OpponentD2H  OpponentD2L
0         31         1           63         0.0     4.0           2.0  0.43078735  0.510272508                0                   1.0  0.362521694  0.562039199,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  OpponentVisible OpponentFirstSeenWall  OpponentD2H  OpponentD2L
0         21         0           42         1.0     4.0           1.0  0.906108861  0.329014745                0                   1.0  0.893476157  0.131553932,    SessionID  PlayerID  GlmPlayerID  ChooseHigh WallSep FirstSeenWall          D2H          D2L  OpponentVisible OpponentFirstSeenWall  OpponentD2H  OpponentD2L
0         31         0           62         0.0     2.0           2.0  0.493552107  0.321920475                0                   1.0  0.487531745  0.607921912,    SessionID  PlayerID  GlmPlaye

IntProgress(value=0, description='Fitting models', max=400)

d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_vars = ran_vars.applymap(
d:\Users\Tom\miniconda3\envs\octagon_analysis\Lib\site-packages\pymer4\models\Lmer.py:733: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  ran_v

CV data saved to:  ..\data\CV_results_social_randomintercepts_allinteractions_400.pickle
CV models saved to:  ..\models\CV_models_social_randomintercepts_allinteractions_400.pickle
